In [1]:
# import
import zipfile


In [5]:
# extract zip file quickly and in gitignore'd folder

zip_file_path = "./data/archive.zip"

# Step 2: Extract the downloaded zip file
extracted_folder_path = "./data/RoadDetectionFiles"
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

In [6]:
# Define paths

test_image = './data/RoadDetectionFiles/Test/image'
train_image = './data/RoadDetectionFiles/Train/image'
validation_image = './data/RoadDetectionFiles/Validation/image'

test_centerline = './data/RoadDetectionFiles/Test/centerline'
train_centerline = './data/RoadDetectionFiles/Train/centerline'
validation_centerline = './data/RoadDetectionFiles/Validation/centerline'

test_label = './data/RoadDetectionFiles/Test/label'
train_label = './data/RoadDetectionFiles/Train/label'
validation_label = './data/RoadDetectionFiles/Validation/label'

In [ ]:
# chat gpt code to design a CasNet architecture to simultaneously handle road detection and centerline extraction tasks

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define directories
train_image_dir = './data/RoadDetectionFiles/Train/image'
validation_image_dir = './data/RoadDetectionFiles/Validation/image'
test_image_dir = './data/RoadDetectionFiles/Test/image'

train_centerline_dir = './data/RoadDetectionFiles/Train/centerline'
validation_centerline_dir = './data/RoadDetectionFiles/Validation/centerline'
test_centerline_dir = './data/RoadDetectionFiles/Test/centerline'

# Image dimensions
image_height = 128
image_width = 128
batch_size = 32
num_epochs = 10

# Load the data
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_image_generator = train_datagen.flow_from_directory(
        train_image_dir,
        target_size=(image_height, image_width),
        batch_size=batch_size,
        class_mode=None,  # No class labels
        color_mode='grayscale')  # Assuming grayscale images

train_centerline_generator = train_datagen.flow_from_directory(
        train_centerline_dir,
        target_size=(image_height, image_width),
        batch_size=batch_size,
        class_mode=None,  # No class labels
        color_mode='grayscale')  # Assuming grayscale images

validation_image_generator = validation_datagen.flow_from_directory(
        validation_image_dir,
        target_size=(image_height, image_width),
        batch_size=batch_size,
        class_mode=None,  # No class labels
        color_mode='grayscale')  # Assuming grayscale images

validation_centerline_generator = validation_datagen.flow_from_directory(
        validation_centerline_dir,
        target_size=(image_height, image_width),
        batch_size=batch_size,
        class_mode=None,  # No class labels
        color_mode='grayscale')  # Assuming grayscale images

test_image_generator = test_datagen.flow_from_directory(
        test_image_dir,
        target_size=(image_height, image_width),
        batch_size=batch_size,
        class_mode=None,  # No class labels
        color_mode='grayscale')  # Assuming grayscale images

test_centerline_generator = test_datagen.flow_from_directory(
        test_centerline_dir,
        target_size=(image_height, image_width),
        batch_size=batch_size,
        class_mode=None,  # No class labels
        color_mode='grayscale')  # Assuming grayscale images

# Combine image and centerline generators into one
train_generator = zip(train_image_generator, train_centerline_generator)
validation_generator = zip(validation_image_generator, validation_centerline_generator)
test_generator = zip(test_image_generator, test_centerline_generator)

# Define CasNet architecture
input_img = layers.Input(shape=(image_height, image_width, 1))

# Road Detection Branch
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
road_output = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same', name='road_output')(x)

# Centerline Extraction Branch
y = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
y = layers.MaxPooling2D((2, 2), padding='same')(y)
y = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(y)
y = layers.MaxPooling2D((2, 2), padding='same')(y)
y = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(y)
centerline_output = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same', name='centerline_output')(y)

# Combine both branches
combined_output = layers.Concatenate()([road_output, centerline_output])

model = models.Model(inputs=input_img, outputs=combined_output)

# Compile the model
model.compile(optimizer='adam',
              loss={'road_output': 'binary_crossentropy', 'centerline_output': 'binary_crossentropy'},
              metrics=['accuracy'])

# Train the model
history = model.fit(
      train_generator,
      epochs=num_epochs,
      steps_per_epoch=len(train_image_generator),  # Adjust steps per epoch based on your data size
      validation_data=validation_generator,
      validation_steps=len(validation_image_generator))  # Adjust validation steps based on your data size

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator, verbose=2, steps=len(test_image_generator))  # Adjust steps based on your data size
print('\nTest accuracy:', test_acc)


In [ ]:
# chat gpt prompted code to do the same thing but this time U-Net

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define directories
train_image_dir = './data/RoadDetectionFiles/Train/image'
validation_image_dir = './data/RoadDetectionFiles/Validation/image'
test_image_dir = './data/RoadDetectionFiles/Test/image'

train_centerline_dir = './data/RoadDetectionFiles/Train/centerline'
validation_centerline_dir = './data/RoadDetectionFiles/Validation/centerline'
test_centerline_dir = './data/RoadDetectionFiles/Test/centerline'

# Image dimensions
image_height = 128
image_width = 128
batch_size = 32
num_epochs = 10

# Load the data
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_image_generator = train_datagen.flow_from_directory(
        train_image_dir,
        target_size=(image_height, image_width),
        batch_size=batch_size,
        class_mode=None,  # No class labels
        color_mode='grayscale')  # Assuming grayscale images

train_centerline_generator = train_datagen.flow_from_directory(
        train_centerline_dir,
        target_size=(image_height, image_width),
        batch_size=batch_size,
        class_mode=None,  # No class labels
        color_mode='grayscale')  # Assuming grayscale images

validation_image_generator = validation_datagen.flow_from_directory(
        validation_image_dir,
        target_size=(image_height, image_width),
        batch_size=batch_size,
        class_mode=None,  # No class labels
        color_mode='grayscale')  # Assuming grayscale images

validation_centerline_generator = validation_datagen.flow_from_directory(
        validation_centerline_dir,
        target_size=(image_height, image_width),
        batch_size=batch_size,
        class_mode=None,  # No class labels
        color_mode='grayscale')  # Assuming grayscale images

test_image_generator = test_datagen.flow_from_directory(
        test_image_dir,
        target_size=(image_height, image_width),
        batch_size=batch_size,
        class_mode=None,  # No class labels
        color_mode='grayscale')  # Assuming grayscale images

test_centerline_generator = test_datagen.flow_from_directory(
        test_centerline_dir,
        target_size=(image_height, image_width),
        batch_size=batch_size,
        class_mode=None,  # No class labels
        color_mode='grayscale')  # Assuming grayscale images

# Combine image and centerline generators into one
train_generator = zip(train_image_generator, train_centerline_generator)
validation_generator = zip(validation_image_generator, validation_centerline_generator)
test_generator = zip(test_image_generator, test_centerline_generator)

# Define U-Net architecture
def unet(input_size=(image_height, image_width, 1)):
    inputs = layers.Input(input_size)
    
    # Encoder
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = layers.Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = layers.Conv2D(512, 3, activation='relu', padding='same')(conv4)
    drop4 = layers.Dropout(0.5)(conv4)
    pool4 = layers.MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = layers.Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = layers.Conv2D(1024, 3, activation='relu', padding='same')(conv5)
    drop5 = layers.Dropout(0.5)(conv5)

    # Decoder
    up6 = layers.Conv2D(512, 2, activation='relu', padding='same')(layers.UpSampling2D(size=(2, 2))(drop5))
    merge6 = layers.concatenate([drop4, up6], axis=3)
    conv6 = layers.Conv2D(512, 3, activation='relu', padding='same')(merge6)
    conv6 = layers.Conv2D(512, 3, activation='relu', padding='same')(conv6)

    up7 = layers.Conv2D(256, 2, activation='relu', padding='same')(layers.UpSampling2D(size=(2, 2))(conv6))
    merge7 = layers.concatenate([conv3, up7], axis=3)
    conv7 = layers.Conv2D(256, 3, activation='relu', padding='same')(merge7)
    conv7 = layers.Conv2D(256, 3, activation='relu', padding='same')(conv7)

    up8 = layers.Conv2D(128, 2, activation='relu', padding='same')(layers.UpSampling2D(size=(2, 2))(conv7))
    merge8 = layers.concatenate([conv2, up8], axis=3)
    conv8 = layers.Conv2D(128, 3, activation='relu', padding='same')(merge8)
    conv8 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv8)

    up9 = layers.Conv2D(64, 2, activation='relu', padding='same')(layers.UpSampling2D(size=(2, 2))(conv8))
    merge9 = layers.concatenate([conv1, up9], axis=3)
    conv9 = layers.Conv2D(64, 3, activation='relu', padding='same')(merge9)
    conv9 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv9)

    # Output layers
    road_output = layers.Conv2D(1, (1, 1), activation='sigmoid', name='road_output')(conv9)
    centerline_output = layers.Conv2D(1, (1, 1), activation='sigmoid', name='centerline_output')(conv9)

    model = models.Model(inputs=[inputs], outputs=[road_output, centerline_output])
    
    return model

# Create U-Net model
model = unet()

# Compile the model
model.compile(optimizer='adam',
              loss={'road_output': 'binary_crossentropy', 'centerline_output': 'binary_crossentropy'},
              metrics={'road_output': 'accuracy', 'centerline_output': 'accuracy'})

# Train the model
history = model.fit(
      train_generator,
      epochs=num_epochs,
      steps_per_epoch=len(train_image_generator),  # Adjust steps per epoch based on your data size
      validation_data=validation_generator,
      validation_steps=len(validation_image_generator))  # Adjust validation steps based on your data size

# Evaluate the model on the test set
test_loss, road_test_loss, centerline_test_loss, road_test_acc, centerline_test_acc = model.evaluate(test_generator, verbose=2, steps=len(test_image_generator))  # Adjust steps based on your data size
print('\nTest Road Detection Accuracy:', road_test_acc)
print('Test Centerline Extraction Accuracy:', centerline_test_acc)
